<a href="https://colab.research.google.com/github/LilySu/BetterBusinessByReview/blob/master/Tally_EndPoint_Refactoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
# !pip install scattertext
# !pip install flask_cors
# !pip install python-decouple

     |████████████████████████████████| 6.8MB 2.7MB/s 
  Created wheel for python-decouple: filename=python_decouple-3.3-cp36-none-any.whl size=9024 sha256=39e1a4a0684be298818c9235f7d4cbcc5dac1ab72a58c9870725c9ba9d572ef3
  Stored in directory: /root/.cache/pip/wheels/f4/9b/79/5eec9ea205382552b014e749cb8a2b4ce0d6836a6f4db5ac0c
Successfully built python-decouple


In [ ]:
# !pip install -U spacy
# !python -m spacy download en_core_web_sm


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
from flask import Flask, render_template, request, jsonify
import json
import warnings
import pandas as pd
import spacy
import scattertext as st
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
import requests
from flask_cors import CORS
from decouple import config

In [ ]:
nlp = spacy.load("en_core_web_sm")

# Code


In [ ]:
base_url = "https://www.yelp.com/biz/" # add business id
api_url = "/review_feed?sort_by=date_desc&start="
bid = 'Kt4NZ9IXvRCHI8qvV0h8qg'

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = json.loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))

s = Scraper()
s.scrape()
df = s.data
df = df.sample(100)

df.head(2)

,0,1,2
18,\n 6/7/2016\n,This place is my JAM. I always try to make a s...,5.0 star rating
10,\n 5/26/2018\n,I was visiting from out of town and wanted to ...,5.0 star rating


In [ ]:
corpus = st.CorpusFromPandas(df, 
                          category_col=2, 
                          text_col=1,
                          nlp=nlp).build()

term_freq_df = corpus.get_term_freq_df()
term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')

term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')
dh = term_freq_df.sort_values(by= 'highratingscore', ascending = False)
dh = dh[['highratingscore', 'poorratingscore']]
dh = dh.reset_index(drop=False)

In [ ]:
dh = dh.rename(columns={'highratingscore':'score'})
dh = dh.drop(columns='poorratingscore')
positive_df = dh.head(10)
negative_df = dh.tail(10)
# word_df = pd.concat([positive_df, negative_df])
# word_df

In [ ]:
results = {'positive': [{'term': pos_term, 'score': pos_score} for pos_term, pos_score in zip(positive_df['term'], positive_df['score'])], 'negative': [{'term': neg_term, 'score': neg_score} for neg_term, neg_score in zip(negative_df['term'], negative_df['score'])]}

In [ ]:
results

{'negative': [{'score': 0.03301130523201978, 'term': 'pour over'},
  {'score': 0.03301130523201978, 'term': 'seats'},
  {'score': 0.02510666224208863, 'term': "n't"},
  {'score': 0.017851582099353724, 'term': 'inside'},
  {'score': 0.017851582099353724, 'term': 'counter'},
  {'score': 0.016076611300342436, 'term': 'pour'},
  {'score': 0.012149792771618761, 'term': 'an'},
  {'score': 0.004435059520226214, 'term': 'over'},
  {'score': 0.0, 'term': 'she'},
  {'score': 0.0, 'term': 'while'}],
 'positive': [{'score': 1.0, 'term': "'ve"},
  {'score': 0.9930754317101049, 'term': 'also'},
  {'score': 0.9921167401729372, 'term': "i 've"},
  {'score': 0.972670377645104, 'term': 'best'},
  {'score': 0.9665201105289263, 'term': 'nice'},
  {'score': 0.9628389324069242, 'term': 'and i'},
  {'score': 0.9615896187531767, 'term': 'super'},
  {'score': 0.9591165396694632, 'term': 'staff'},
  {'score': 0.9591165396694632, 'term': 'from'},
  {'score': 0.9545264154685009, 'term': 'great'}]}